In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kendalltau, pearsonr, spearmanr

def spearsonr_pval(x,y):
    return spearmanr(x,y)[1]

data_full = pd.read_csv("RVB-extra-info-M-F-fix-08092022.csv")
data_a = data_full[data_full["Treatmentgroup"].isin(["Group1C","Group3E"])]

data_not_restore = pd.read_csv("RVB-mass-not-restored.csv")
data_not_restore_list = data_not_restore.filename.to_list()
data_not_restore_list.insert(0, "MouseID")

# this is metadata info, which is extra baylor information.
# careful for the range, [a,b] mean a <= X < b. Also python is 0 based index.
data_M = data_a.iloc[:, np.r_[0, 2:9]]
data_M = data_M.set_index('MouseID')
# this is metabolites, which is features.
data_F = data_a[data_not_restore_list]
data_F = data_F.set_index('MouseID')

# get the two part of column names: metadata colum name and features column name.
M_column_list = data_M.head(0)
print(M_column_list)
F_column_list = data_F.head(0)
print(F_column_list)

df_col_list = []
df_corr_list = []
df_pval_list = []

for m_col in M_column_list:
    for f_col in F_column_list:
        col_1 = data_M[m_col]
        col_2 = data_F[f_col]
        corr_col_1_2 = col_1.corr(col_2, method='spearman')
        corr_pval_col_1_2 = col_1.corr(col_2, method=spearsonr_pval)
        df_col_list.append(m_col + "-" + f_col)
        df_corr_list.append(corr_col_1_2)
        df_pval_list.append(corr_pval_col_1_2)

print(df_col_list)
print(df_corr_list)
print(df_pval_list)

import statsmodels.api
dr_fdr_pval = statsmodels.stats.multitest.fdrcorrection(df_pval_list, method='indep')
df_final = pd.DataFrame(list(zip(df_col_list, df_corr_list, df_pval_list, dr_fdr_pval[0], dr_fdr_pval[1])), columns=['col_name','corr', 'corr_pval', 'FDR_rejected', 'FDR_pval'])

df_final.to_csv('RVB-no-treat-not-restored-corr-pval.csv', index=False)

Empty DataFrame
Columns: [Heart.Weight..Body.Weight, Liver.Weight..Body.Weight, Ejection.Fraction, PR.Interval..s., QTc..s., P.Amplitude..V., CD3.CD8.IFNg.]
Index: []
Empty DataFrame
Columns: [X105.07_2.767, X111.044_2.768, X137.046_0.35, X150.059_0.3, X200.165_3.207, X249.112_2.768, X267.123_2.767, X268.105_0.35, X269.088_0.35, X278.248_3.1, X280.264_3.159, X285.133_2.768, X303.648_2.884, X306.279_2.989, X306.279_3.212, X369.17_2.767, X387.181_2.767, X404.207_2.768, X406.154_2.765, X415.14_2.768, X430.223_2.768, X446.218_2.884, X475.989_2.164, X476.24_2.164, X486.098_2.765, X489.228_2.768, X511.21_3.171, X528.311_2.928, X570.357_2.934, X599.241_2.765, X608.227_2.768, X637.306_3.32, X654.332_3.32, X790.38_2.768, X792.329_2.764]
Index: []

[0 rows x 35 columns]
['Heart.Weight..Body.Weight-X105.07_2.767', 'Heart.Weight..Body.Weight-X111.044_2.768', 'Heart.Weight..Body.Weight-X137.046_0.35', 'Heart.Weight..Body.Weight-X150.059_0.3', 'Heart.Weight..Body.Weight-X200.165_3.207', 'Heart.Weigh